<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
import random 
import re
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.layers import Dense, Embedding, Flatten 
# MultiHeadAttention

In [ ]:
## 1) Sentences will be tokenized using the TextVectorization 
## 2) 12.5 % of the tokens will be replaced with the [MASK] token. Substituded tokens will be 
## chosen randomly. To select the random tokens, random sampling will be chosen 

## 3) segment id will be determined at the second step
## 4) positional embeddings will be directly taken from the article

In [2]:
max_length =  128  ## 256
corpus_length = 128   ## 30000
embedding_dimension = 128   ## 128
vocab_size = 128  ## 30000




In [3]:
trial_sentences = [
    'my name is john',
    'london is the capital is the england',
    'weather is cold at the northern hemisphere of the world',
    'today, i will go the school'
]

longest_sentence = []
for i in trial_sentences:
    longest_sentence.append(len(i.split()))
    
print(np.max(longest_sentence))

longest_sentence = np.max(longest_sentence)

10


In [4]:
def masking_procedure (text):
    
    tf_tokenizer = Tokenizer()
    tf_tokens=tf_tokenizer.fit_on_texts(text)

    corpus = tf_tokenizer.word_index  ### TOKEN SET WITH TOKEN IDS WITHOUT THE SPECIAL TOKENS 
    extended_corpus = corpus.copy()   ### TOKEN SET WITH SPECIAL TOKENS

    extended_corpus['MASK'] = len(corpus)+1

    corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
    for i,j in enumerate(corpus):
        corpus_tokens.append(j)
#         print(i,j)

        
    extended_tokens = []
    for i,j in  enumerate(extended_corpus):
    #     print(i,j)
        extended_tokens.append(j)
    # print('maskelenmis_corpus')
    # print('-------')
    subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12)) ## RANDOMLY SELECTED TOKENS TO BE MASKED
#     print(subs_tokens)

    sequences = tf_tokenizer.texts_to_sequences(text)
    padded_sequences= pad_sequences(sequences)
    padded_sequences

    substitute_padded_sequences= padded_sequences.copy()

    subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

    for i in subs_tokens:
        subs_tokens_ids.append(tf_tokenizer.word_index[i])
    
    
    mask_token_id = 20
#     print(subs_tokens_ids)



    for i in range(len(extended_tokens)):
        if i == (subs_tokens_ids[0]-1   ):
            extended_tokens[i] = '[MASK]'
    
        if i == (subs_tokens_ids[1]-1):
            extended_tokens[i] = '[MASK]'
        
        
        
    maskeli = []
    for i in range(substitute_padded_sequences.shape[0]):
        for j in range(substitute_padded_sequences.shape[1]):
            if substitute_padded_sequences[i][j] != 0:
                x = (substitute_padded_sequences[i][j]  )
                maskeli.append("".join(extended_tokens[x-1]))
                
                
    maskeli_2 = []
    for i in (substitute_padded_sequences):
        maskeli_2.append('\n')
        for j in range(len(i)):
        
            if (i[j] !=0):
#             print(i[j])
                maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
    y= " ".join(maskeli_2)

    y = y.split('\n')
    y = np.delete(y,0,0)
    y
    
    segment_ids = substitute_padded_sequences.copy()

    for i in range(segment_ids.shape[0]):
        for j in range(segment_ids.shape[1]):
            if (segment_ids[i][j] != 0):
                segment_ids[i][j] = 1
    segment_ids = np.array(segment_ids)
    
    return (y , extended_tokens, segment_ids,substitute_padded_sequences)

In [5]:
masked_text, masked_tokens,segment_ids , masked_sequence = masking_procedure(trial_sentences)

In [7]:
MAX_LEN = 256
EMBED_DIM = 128



def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc


positional_embeddings = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)